# Word Search II [hard]

Source: https://leetcode.com/problems/word-search-ii/description/

Each word must be constructed from letters of sequentially adjacent cells, where adjacent cells are horizontally or vertically neighboring. The same letter cell may not be used more than once in a word. 

## Example 1:

![](images/ws2_ex1.png)

Input: board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], words = ["oath","pea","eat","rain"]

Output: ["eat","oath"]

## Example 2:

![](images/ws2_ex2.png)

Input: board = [["a","b"],["c","d"]], words = ["abcb"]

Output: []
 
## Constraints:

- m == board.length
- n == board[i].length
- 1 <= m, n <= 12
- board[i][j] is a lowercase English letter.
- 1 <= words.length <= 3 * 10^4
- 1 <= words[i].length <= 10
- words[i] consists of lowercase English letters.
- All the strings of words are unique.

In [17]:
from typing import List, Optional

# This doesn't work on the big word set, we have MxN starting locations times 10 max word length times 30000 max words
class Solution1:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        def backtrack(i:int, j:int, word:str, w_pos:int) -> bool:
            # Base case, check if we found the word
            if w_pos >= len(word):
                return True

            # Check for out of bounds
            if i < 0 or i >= len(board) or j < 0 or j >= len(board[i]):
                return False

            # Check if we have a character match
            if board[i][j] != word[w_pos]:
                return False

            # Mark current position as used
            temp = board[i][j]
            board[i][j] = "*"

            # Step to next
            result = backtrack(i+1,j,word,w_pos+1) or backtrack(i-1,j,word,w_pos+1) or backtrack(i,j+1,word,w_pos+1) or backtrack(i,j-1,word,w_pos+1)

            # Return board to normal state
            board[i][j] = temp

            return result
            
        
        found_words = []

        for word in words:
            found = False
            for i in range(len(board)):
                for j in range(len(board[i])):
                    if backtrack(i, j, word, 0):
                        # We have a match
                        found_words.append(word)

                        # Break out of this nested loop and continue to next word
                        found = True
                        break

                if found:
                    break
                
        return found_words

# Try a Trie implementation
class TrieNode:
    def __init__(self, value:str, seen:bool = False, parent:Optional["TrieNode"] = None, children:list["TrieNode"] = []):
        self.value: str = value
        self.is_seen = seen        # Flag to indicate it was seen
        self.parent: Optional['TrieNode'] = parent
        # Use hashmap for better performance and lookup
        self.children: dict[str, 'TrieNode'] = {tn.value: tn for tn in children}

    def add_child(self, value:str) -> 'TrieNode':
        # Prevent duplicates
        if self.has_child(value):
            return self.children.get(value)

        child = TrieNode(value, parent = self)
        
        self.children[child.value] = child

        return child

    def has_child(self, value:str) -> bool:
        return value in self.children

    def get_child(self, value:str) -> Optional['TrieNode']:
        if not self.has_child(value):
            return None
        
        return self.children.get(value)

    def drop_child(self, value:str):
        child = self.children[value]
        child.parent = None
        del self.children[value]

    def recurse_value(self) -> str:
        return self.value + self.recurse_value()

    def seen(self):
        self.is_seen = True

    def __str__(self):
        return self.value


class Trie:
    def __init__(self):
        self.root = TrieNode('', seen = True)

    def add_word(self, word:str):
        node = self.root
        for c in word:
            node = node.add_child(c)

    def has_word(self, word:str) -> bool:
        """Accepts partial matches"""

        idx = 0 
        node = self.root
        while idx < len(word):
            node = node.get_child(word[idx])

            if node is None:
                return False

            # Step
            idx += 1
            
        return True

    def seen(self, word:str) -> bool:
        """Return False if the word is not in the Trie, else return True"""
        node = self.root
        for c in word:
            node = node.get_child(c)

            if node is None:
                return False

            node.seen()

        return True
        
    def _reduce(self, node:TrieNode) -> list[str]:
        # Base case: Find a tail
        if len(node.children.values()) == 0:
            return [str(node)]

        words = []
        for child in node.children.values():
            # Only add seen children nodes
            if child.is_seen:
                words += self._reduce(child) # concatenate the list of words

        # Add this node's value to each word
        words = [str(node) + word for word in words]

        return words

    def reduce(self) -> list[str]:
        """Reduce Trie into a list of seen words"""
        return self._reduce(self.root)
        
    def __str__(self):
        result = []
        
        def print_node(node, depth=0, prefix="", only_seen=False):
            # Add the current node to the result
            indent = "  " * depth
            result.append(f"{prefix}{indent}{node.value}")
            
            # Sort children for consistent output
            sorted_children = sorted(node.children.items())
            seen_children = [child for child in sorted_children if child[1].is_seen]

            
            # Print all children
            for i, (key, child) in enumerate(seen_children):
                is_last = i == len(seen_children) - 1
                new_prefix = prefix + ("└── " if is_last else "├── ")
                next_prefix = prefix + ("    " if is_last else "│   ")
                print_node(child, depth + 1, new_prefix)
        
        print_node(self.root, only_seen=True)
        return "\n".join(result)


# Trie implementation allows use to: 
# 1. only search the entire board and all paths up to N word length once 
# 2. handle multiple words at the same time
# 3. cancel out a search path early if no words fit
# 
# Time Complexity: O(AxBx4^N) where AxB is dimension of the grid, and N is max length of the words
# Space Complexity: O(N*Max words)

class Solution2:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # Create and populate our Trie
        trie = Trie()
        for word in words:
            trie.add_word(word)

        # Recursively search each path for matching words
        def backtrack(i, j, word:str, max_depth = 10):
            # Drop out if we hit depth limit
            if len(word) > max_depth:
                return

            # Drop out if word isn't in the trie
            if not trie.has_word(word + board[i][j]):
                return

            # Mark this word as seen
            trie.seen(word + board[i][j])

            # Check generate moves
            up, down, left, right = (i+1, j), (i-1, j), (i, j-1), (i, j+1)
            possible_moves = [up, down, left, right]

            # Validate our moves
            valid_moves = []
            for move in possible_moves:
                if 0 <= move[0] < len(board) and 0 <= move[1] < len(board[move[0]]):
                    if board[move[0]][move[1]] is not None:
                        valid_moves.append(move)

            # Mark current position on board as None
            temp = board[i][j]
            board[i][j] = None

            # Recursive step
            for move in valid_moves:
                backtrack(move[0], move[1], word + temp)

            # Reset board to original state
            board[i][j] = temp
            

        # Check all possible paths and mark any words in the 
        for i in range(len(board)):
            for j in range(len(board[i])):
                backtrack(i, j, '')

        # Now reduce our Trie and get only the seen words
        return trie.reduce()


In [20]:
test_cases = [
    {
        "board": [
            ["o","a","a","n"],
            ["e","t","a","e"],
            ["i","h","k","r"],
            ["i","f","l","v"]
        ],
        "words": ["oath","pea","eat","rain"],
        "expected": ["eat","oath"]
    },
    {
        "board": [
            ["a", "b"],
            ["c", "d"]
        ],
        "words": ["abcb"],
        "expected": []
    },
    {
        "board": [
            ["b","a","b","a","b","a","b","a","b","a"],
            ["a","b","a","b","a","b","a","b","a","b"],
            ["b","a","b","a","b","a","b","a","b","a"],
            ["a","b","a","b","a","b","a","b","a","b"],
            ["b","a","b","a","b","a","b","a","b","a"],
            ["a","b","a","b","a","b","a","b","a","b"],
            ["b","a","b","a","b","a","b","a","b","a"],
            ["a","b","a","b","a","b","a","b","a","b"],
            ["b","a","b","a","b","a","b","a","b","a"],
            ["a","b","a","b","a","b","a","b","a","b"]
            ],
            "words": ["ababababaa","ababababab","ababababac","ababababad","ababababae","ababababaf","ababababag","ababababah","ababababai","ababababaj","ababababak","ababababal","ababababam","ababababan","ababababao","ababababap","ababababaq","ababababar","ababababas","ababababat","ababababau","ababababav","ababababaw","ababababax","ababababay","ababababaz","ababababba","ababababbb","ababababbc","ababababbd","ababababbe","ababababbf","ababababbg","ababababbh","ababababbi","ababababbj","ababababbk","ababababbl","ababababbm","ababababbn","ababababbo","ababababbp","ababababbq","ababababbr","ababababbs","ababababbt","ababababbu","ababababbv","ababababbw","ababababbx","ababababby","ababababbz","ababababca","ababababcb","ababababcc","ababababcd","ababababce","ababababcf","ababababcg","ababababch","ababababci","ababababcj","ababababck","ababababcl","ababababcm","ababababcn","ababababco","ababababcp","ababababcq","ababababcr","ababababcs","ababababct","ababababcu","ababababcv","ababababcw","ababababcx","ababababcy","ababababcz","ababababda","ababababdb","ababababdc","ababababdd","ababababde","ababababdf","ababababdg","ababababdh","ababababdi","ababababdj","ababababdk","ababababdl","ababababdm","ababababdn","ababababdo","ababababdp","ababababdq","ababababdr","ababababds","ababababdt","ababababdu","ababababdv","ababababdw","ababababdx","ababababdy","ababababdz","ababababea","ababababeb","ababababec","ababababed","ababababee","ababababef","ababababeg","ababababeh","ababababei","ababababej","ababababek","ababababel","ababababem","ababababen","ababababeo","ababababep","ababababeq","ababababer","ababababes","ababababet","ababababeu","ababababev","ababababew","ababababex","ababababey","ababababez","ababababfa","ababababfb","ababababfc","ababababfd","ababababfe","ababababff","ababababfg","ababababfh","ababababfi","ababababfj","ababababfk","ababababfl","ababababfm","ababababfn","ababababfo","ababababfp","ababababfq","ababababfr","ababababfs","ababababft","ababababfu","ababababfv","ababababfw","ababababfx","ababababfy","ababababfz","ababababga","ababababgb","ababababgc","ababababgd","ababababge","ababababgf","ababababgg","ababababgh","ababababgi","ababababgj","ababababgk","ababababgl","ababababgm","ababababgn","ababababgo","ababababgp","ababababgq","ababababgr","ababababgs","ababababgt","ababababgu","ababababgv","ababababgw","ababababgx","ababababgy","ababababgz","ababababha","ababababhb","ababababhc","ababababhd","ababababhe","ababababhf","ababababhg","ababababhh","ababababhi","ababababhj","ababababhk","ababababhl","ababababhm","ababababhn","ababababho","ababababhp","ababababhq","ababababhr","ababababhs","ababababht","ababababhu","ababababhv","ababababhw","ababababhx","ababababhy","ababababhz","ababababia","ababababib","ababababic","ababababid","ababababie","ababababif","ababababig","ababababih","ababababii","ababababij","ababababik","ababababil","ababababim","ababababin","ababababio","ababababip","ababababiq","ababababir","ababababis","ababababit","ababababiu","ababababiv","ababababiw","ababababix","ababababiy","ababababiz","ababababja","ababababjb","ababababjc","ababababjd","ababababje","ababababjf","ababababjg","ababababjh","ababababji","ababababjj","ababababjk","ababababjl","ababababjm","ababababjn","ababababjo","ababababjp","ababababjq","ababababjr","ababababjs","ababababjt","ababababju","ababababjv","ababababjw","ababababjx","ababababjy","ababababjz","ababababka","ababababkb","ababababkc","ababababkd","ababababke","ababababkf","ababababkg","ababababkh","ababababki","ababababkj","ababababkk","ababababkl","ababababkm","ababababkn","ababababko","ababababkp","ababababkq","ababababkr","ababababks","ababababkt","ababababku","ababababkv","ababababkw","ababababkx","ababababky","ababababkz","ababababla","abababablb","abababablc","ababababld","abababable","abababablf","abababablg","abababablh","ababababli","abababablj","abababablk","ababababll","abababablm","ababababln","abababablo","abababablp","abababablq","abababablr","ababababls","abababablt","abababablu","abababablv","abababablw","abababablx","abababably","abababablz","ababababma","ababababmb","ababababmc","ababababmd","ababababme","ababababmf","ababababmg","ababababmh","ababababmi","ababababmj","ababababmk","ababababml","ababababmm","ababababmn","ababababmo","ababababmp","ababababmq","ababababmr","ababababms","ababababmt","ababababmu","ababababmv","ababababmw","ababababmx","ababababmy","ababababmz","ababababna","ababababnb","ababababnc","ababababnd","ababababne","ababababnf","ababababng","ababababnh","ababababni","ababababnj","ababababnk","ababababnl","ababababnm","ababababnn","ababababno","ababababnp","ababababnq","ababababnr","ababababns","ababababnt","ababababnu","ababababnv","ababababnw","ababababnx","ababababny","ababababnz","ababababoa","ababababob","ababababoc","ababababod","ababababoe","ababababof","ababababog","ababababoh","ababababoi","ababababoj","ababababok","ababababol","ababababom","ababababon","ababababoo","ababababop","ababababoq","ababababor","ababababos","ababababot","ababababou","ababababov","ababababow","ababababox","ababababoy","ababababoz","ababababpa","ababababpb","ababababpc","ababababpd","ababababpe","ababababpf","ababababpg","ababababph","ababababpi","ababababpj","ababababpk","ababababpl","ababababpm","ababababpn","ababababpo","ababababpp","ababababpq","ababababpr","ababababps","ababababpt","ababababpu","ababababpv","ababababpw","ababababpx","ababababpy","ababababpz","ababababqa","ababababqb","ababababqc","ababababqd","ababababqe","ababababqf","ababababqg","ababababqh","ababababqi","ababababqj","ababababqk","ababababql","ababababqm","ababababqn","ababababqo","ababababqp","ababababqq","ababababqr","ababababqs","ababababqt","ababababqu","ababababqv","ababababqw","ababababqx","ababababqy","ababababqz","ababababra","ababababrb","ababababrc","ababababrd","ababababre","ababababrf","ababababrg","ababababrh","ababababri","ababababrj","ababababrk","ababababrl","ababababrm","ababababrn","ababababro","ababababrp","ababababrq","ababababrr","ababababrs","ababababrt","ababababru","ababababrv","ababababrw","ababababrx","ababababry","ababababrz","ababababsa","ababababsb","ababababsc","ababababsd","ababababse","ababababsf","ababababsg","ababababsh","ababababsi","ababababsj","ababababsk","ababababsl","ababababsm","ababababsn","ababababso","ababababsp","ababababsq","ababababsr","ababababss","ababababst","ababababsu","ababababsv","ababababsw","ababababsx","ababababsy","ababababsz","ababababta","ababababtb","ababababtc","ababababtd","ababababte","ababababtf","ababababtg","ababababth","ababababti","ababababtj","ababababtk","ababababtl","ababababtm","ababababtn","ababababto","ababababtp","ababababtq","ababababtr","ababababts","ababababtt","ababababtu","ababababtv","ababababtw","ababababtx","ababababty","ababababtz","ababababua","ababababub","ababababuc","ababababud","ababababue","ababababuf","ababababug","ababababuh","ababababui","ababababuj","ababababuk","ababababul","ababababum","ababababun","ababababuo","ababababup","ababababuq","ababababur","ababababus","ababababut","ababababuu","ababababuv","ababababuw","ababababux","ababababuy","ababababuz","ababababva","ababababvb","ababababvc","ababababvd","ababababve","ababababvf","ababababvg","ababababvh","ababababvi","ababababvj","ababababvk","ababababvl","ababababvm","ababababvn","ababababvo","ababababvp","ababababvq","ababababvr","ababababvs","ababababvt","ababababvu","ababababvv","ababababvw","ababababvx","ababababvy","ababababvz","ababababwa","ababababwb","ababababwc","ababababwd","ababababwe","ababababwf","ababababwg","ababababwh","ababababwi","ababababwj","ababababwk","ababababwl","ababababwm","ababababwn","ababababwo","ababababwp","ababababwq","ababababwr","ababababws","ababababwt","ababababwu","ababababwv","ababababww","ababababwx","ababababwy","ababababwz","ababababxa","ababababxb","ababababxc","ababababxd","ababababxe","ababababxf","ababababxg","ababababxh","ababababxi","ababababxj","ababababxk","ababababxl","ababababxm","ababababxn","ababababxo","ababababxp","ababababxq","ababababxr","ababababxs","ababababxt","ababababxu","ababababxv","ababababxw","ababababxx","ababababxy","ababababxz","ababababya","ababababyb","ababababyc","ababababyd","ababababye","ababababyf","ababababyg","ababababyh","ababababyi","ababababyj","ababababyk","ababababyl","ababababym","ababababyn","ababababyo","ababababyp","ababababyq","ababababyr","ababababys","ababababyt","ababababyu","ababababyv","ababababyw","ababababyx","ababababyy","ababababyz","ababababza","ababababzb","ababababzc","ababababzd","ababababze","ababababzf","ababababzg","ababababzh","ababababzi","ababababzj","ababababzk","ababababzl","ababababzm","ababababzn","ababababzo","ababababzp","ababababzq","ababababzr","ababababzs","ababababzt","ababababzu","ababababzv","ababababzw","ababababzx","ababababzy","ababababzz"],
            "expected": []
    }
]

s = Solution2()

for tc in test_cases:
    board = tc.get('board')
    words = tc.get('words')
    expected = tc.get('expected')
    print(f"{s.findWords(board, words) =} == {expected =}")

s.findWords(board, words) =['oath', 'eat'] == expected =['eat', 'oath']
s.findWords(board, words) =[] == expected =[]
s.findWords(board, words) =['ababababab'] == expected =[]
